### Comparison of final outcomes 

This notebook allows to get an idea of the concrete consequences of a harmonization of dates / seasons in the gamma standardization and bias correction between the different issue months and indicators. This comparison is based on the 2024-2025 season and also includes the fix of a bug in the triggers filtering (final step after brute optimization).

In [1]:
%cd ../..

/home/u/amine.barkaoui/anticipatory-action


In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
from config.params import Params

In [4]:
country = "MOZ"

In [5]:
params = Params(iso=country, index='SPI')

### New outcomes

In [58]:
df_merged = pd.read_csv(
    f"{params.data_path}/data/{params.iso}_align_dates/probs/aa_probabilities_triggers_pilots_helpersfixed.csv",
)

In [59]:
df_merged

,district,index,category,window,HR,FR,issue_ready,issue_set,trigger_ready,trigger_set,vulnerability,prob_ready,prob_set
0,Cahora_Bassa,DRYSPELL AM,Mild,Window 2,0.600,0.400000,12.0,1.0,0.35,0.33,NRT,0.33,0.40
1,Cahora_Bassa,DRYSPELL AM,Moderate,Window 2,0.500,0.400000,12.0,1.0,0.34,0.27,NRT,0.27,0.35
2,Cahora_Bassa,DRYSPELL AM,Severe,Window 2,0.500,0.250000,11.0,12.0,0.15,0.31,NRT,0.13,0.25
3,Cahora_Bassa,DRYSPELL FM,Severe,Window 2,0.500,0.400000,9.0,10.0,0.10,0.30,NRT,0.05,0.08
4,Cahora_Bassa,DRYSPELL JF,Mild,Window 1,0.875,0.266667,8.0,9.0,0.05,0.06,NRT,0.22,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,Moamba,SPI NDJ,Moderate,Window 1,0.571,0.200000,7.0,8.0,0.00,0.31,NRT,0.09,0.31
420,Moamba,SPI NDJ,Severe,Window 1,0.600,0.428571,7.0,8.0,0.19,0.18,NRT,0.09,0.22
421,Moamba,SPI ON,Mild,Window 1,0.900,0.187500,5.0,6.0,0.27,0.00,NRT,0.28,0.40
422,Moamba,SPI ON,Moderate,Window 1,0.700,0.181818,5.0,6.0,0.15,0.32,NRT,0.25,0.07


In [60]:
MAPPING = {0: 'NA', 1:'NA', 2: 'NA', 3: 'Set'}

for (d, c, w), case in df_merged.groupby(['district', 'category', 'window']):
    case['ready'] = case['prob_ready'] > case['trigger_ready']
    case['set'] = case['prob_set'] > case['trigger_set']
    case['state'] = 2*case['ready'] + 1*case['set']
    case['state'] = [MAPPING[s] for s in case.state]
    case.loc[case.issue_set == 2, 'state'] = 'NotSetYet'
    state = case.state.max()
    
    df_merged.loc[(df_merged.district == d) & (df_merged.window == w) & (df_merged['category'] == c), 'state'] = state

In [61]:
outcomes_new = pd.DataFrame(columns=["W1-Leve", "W1-Moderado", "W1-Severo", "W2-Leve", "W2-Moderado", "W2-Severo"], index = df_merged['district'].sort_values().unique())
for d, r in outcomes_new.iterrows():
    val = []
    for w in sorted(df_merged['window'].unique()):
        for c in df_merged['category'].unique():
                val.append(df_merged[(df_merged['window']==w) & (df_merged['category']==c) & (df_merged['district']==d)].state.unique()[0])
    outcomes_new.loc[d] = val

In [62]:
outcomes_new.style.format(na_rep="missing so far")

,W1-Leve,W1-Moderado,W1-Severo,W2-Leve,W2-Moderado,W2-Severo
Cahora_Bassa,Set,Set,NA,NA,NA,NA
Caia,Set,Set,Set,NA,NA,NA
Changara,Set,Set,Set,Set,NotSetYet,NA
Chemba,Set,Set,Set,NA,NA,NA
Chibuto,Set,Set,NA,Set,NA,NA
Chicualacuala,Set,NA,NA,NA,Set,Set
Chigubo,NA,NA,NA,Set,Set,Set
Chiure,Set,Set,NA,Set,NA,NA
Funhalouro,NA,NA,NA,Set,Set,NA
Guija,Set,Set,NA,Set,Set,NA


### Previous outcomes

In [40]:
ref = pd.read_csv(
    f"{params.data_path}/data/{params.iso}/probs/aa_probabilities_triggers_pilots.csv",
)

In [41]:
for (d, c, w), case in ref.groupby(['district', 'category', 'window']):
    case['ready'] = case['prob_ready'] > case['trigger_ready']
    case['set'] = case['prob_set'] > case['trigger_set']
    case['state'] = 2*case['ready'] + 1*case['set']
    case['state'] = [MAPPING[s] for s in case.state]
    case.loc[case.issue_set == 2, 'state'] = 'NotSetYet'
    state = case.state.max()
    
    ref.loc[(ref.district == d) & (ref.window == w) & (ref['category'] == c), 'state'] = state

In [42]:
outcomes_old = pd.DataFrame(columns=["W1-Leve", "W1-Moderado", "W1-Severo", "W2-Leve", "W2-Moderado", "W2-Severo"], index = ref['district'].sort_values().unique())
for d, r in outcomes_old.iterrows():
    val = []
    for w in sorted(ref['window'].unique()):
        for c in ['Mild', 'Moderate', 'Severe']:
            val.append(ref[(ref['window']==w) & (ref['category']==c) & (ref['district']==d)].state.unique()[0])
    outcomes_old.loc[d] = val

In [43]:
outcomes_old.style.format(na_rep="missing so far")

,W1-Leve,W1-Moderado,W1-Severo,W2-Leve,W2-Moderado,W2-Severo
Cahora_Bassa,Set,Set,NA,NA,NA,NA
Caia,Set,Set,Set,NA,NA,Set
Changara,Set,Set,Set,Set,NA,NA
Chemba,Set,Set,Set,NA,NA,NA
Chibuto,Set,Set,NA,Set,NA,NA
Chicualacuala,Set,NA,NA,NA,Set,Set
Chigubo,NA,NA,NA,Set,Set,Set
Chiure,Set,Set,NA,NotSetYet,NotSetYet,NotSetYet
Funhalouro,NA,NA,NA,Set,Set,NA
Guija,Set,Set,NA,Set,Set,NA


### More detailed comparison

In [63]:
def compare_new_old_outcomes(old, new):
    comp = pd.DataFrame(columns=["W1-Leve", "W1-Moderado", "W1-Severo", "W2-Leve", "W2-Moderado", "W2-Severo"], index = old.index)
    for d, row in comp.iterrows():
        for col in comp.columns:
       
            if new.loc[d, col] == 'Set' and old.loc[d, col] == 'Set':
                comp.loc[d, col] = 'both set'
            elif new.loc[d, col] == 'NA' and old.loc[d, col] == 'NA':
                comp.loc[d, col] = 'both NA'
            elif new.loc[d, col] == 'Ready' and old.loc[d, col] == 'Ready':
                comp.loc[d, col] = 'both ready'
            elif new.loc[d, col] == 'Set' and old.loc[d, col] == 'NA':
                comp.loc[d, col] = 'now set'
            elif new.loc[d, col] == 'NA' and old.loc[d, col] == 'Set':
                comp.loc[d, col] = 'was set before'
            elif type(new.loc[d, col]) is not str or type(old.loc[d, col]) is not str:
                comp.loc[d, col] = 'missing so far'
            else: 
                comp.loc[d, col] = 'not complete'

    colors = {'now set': 'darkgreen', 'both set': 'mediumseagreen', 'both NA': 'mediumseagreen', 'both ready': 'mediumseagreen', 'was set before': 'coral', 'missing so far': 'burlywood', 'not complete': 'burlywood'}
    return comp.style.map(lambda val: 'background-color: {}'.format(colors.get(val,'')))

In [64]:
compare_new_old_outcomes(outcomes_old, outcomes_new)

,W1-Leve,W1-Moderado,W1-Severo,W2-Leve,W2-Moderado,W2-Severo
Cahora_Bassa,both set,both set,both NA,both NA,both NA,both NA
Caia,both set,both set,both set,both NA,both NA,was set before
Changara,both set,both set,both set,both set,not complete,both NA
Chemba,both set,both set,both set,both NA,both NA,both NA
Chibuto,both set,both set,both NA,both set,both NA,both NA
Chicualacuala,both set,both NA,both NA,both NA,both set,both set
Chigubo,both NA,both NA,both NA,both set,both set,both set
Chiure,both set,both set,both NA,not complete,not complete,not complete
Funhalouro,both NA,both NA,both NA,both set,both set,both NA
Guija,both set,both set,both NA,both set,both set,both NA
